<a href="https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/namd_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Download NAMD
# Download the NAMD file
!wget https://www.ks.uiuc.edu/Research/namd/3.0b2/download/786490/NAMD_3.0b2_Linux-x86_64-multicore-CUDA.tar.gz


In [ ]:
#@title Extract NAMD
!tar -zxf /content/NAMD_3.0b2_Linux-x86_64-multicore-CUDA.tar.gz

In [ ]:
#@title Run NAMD
#@markdown Change the paths to correspond to the files
!/content/NAMD_3.0b2_Linux-x86_64-multicore-CUDA/namd3 +auto-provision +idlepoll +devices 0 /content/drive/MyDrive/namd/B1/B1/namd/step4_equilibration.inp > /content/drive/MyDrive/namd/B1/B1/namd/step4_equilibration.log &

In [ ]:
#@title Restart NAMD from previous stimulation
    #@markdown upload new config file:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
    #@markdown Move uploaded file to folder
!mv /content/step4_equilibration.inp /content/drive/MyDrive/namd/B1/B1/namd
    #@markdown Restart NAMD
!/content/NAMD_3.0b2_Linux-x86_64-multicore-CUDA/namd3 +auto-provision +idlepoll +devices 0 /content/drive/MyDrive/namd/B1/B1/namd/step4_equilibration.inp > /content/drive/MyDrive/namd/B1/B1/namd/step4_equilibration_2.log &

In [ ]:
#@title Extract CHARMM-GUI files (Completed)
# !tar -zxf /content/drive/MyDrive/namd/test/charmm-gui.tgz
# cp -r /content/charmm-gui-8277124303 /content/drive/MyDrive/namd/test
# To unzip file, use !unzip
# Example:
#!unzip /content/drive/MyDrive/namd/BFEE.zip -d /content/drive/MyDrive/namd ## Extract file using for biding free energy estimate

To combine multiples .dcd files, see this notebook: 

Trường hợp file bị lỗi ngắt giữa chừng có thể khắc phục bằng cách:
- Tải xuống file .inp và file .xst
- Mở file .xst bằng Notepad:
  - Sửa output thành tên khác, ví dụ step4 -> step4_1
  - Sửa input thành .restart
  - Mở file .xst xem đang chạy tới step bao nhiêu, sau đó dùng số step muốn chạy trừ đi số step bị dừng. Ví dụ: chạy 500.000 step, file dừng lại ở 245.600 -> 500000-2456000 = 254400. Kéo xuoogns cuối file .inp sửa numsteps và run thành 254400

References:

https://www.youtube.com/watch?v=NHb5s9zDZP8&list=PLN_sQXLvUWhXjGz2c807ijngl-Hh4kp_g&index=20

James C. Phillips, David J. Hardy, Julio D. C. Maia, John E. Stone, João V. Ribeiro, Rafael C. Bernardi, Ronak Buch, Giacomo Fiorin, Jérôme Hénin, Wei Jiang, Ryan McGreevy, Marcelo C. R. Melo, Brian Radak, Robert D. Skeel, Abhishek Singharoy, Yi Wang, Benoît Roux, Aleksei Aksimentiev, Zaida Luthey-Schulten, Laxmikant V. Kalé, Klaus Schulten, Christophe Chipot, and Emad Tajkhorshid. Scalable molecular dynamics on CPU and GPU architectures with NAMD. Journal of Chemical Physics, 153:044130, 2020. (PMC: PMC7395834)

